In [ ]:
#Tool testing code

from dotenv import load_dotenv
from crewai import Agent, Task, Crew, LLM
from crewai.tools import tool
import pandas as pd
from utils.data_fetcher import get_stock_data, get_risk_data
from utils.technical_analysis import add_technical_indicators


load_dotenv()
STOCK_SYMBOL = "TSLA"  

groq_llm_2 = LLM(model="groq/gemma2-9b-it",max_tokens=700)


@tool("Stock Analysis Tool")
def stock_analysis_tool(symbol: str) -> dict:
    """Performs technical analysis using:
    - 50/200-day SMAs 
    - 14-day RSI"""
    try:
        
        data = get_stock_data(symbol)  
        
       
        if len(data) < 200:
            return {"error": f"Need 200 trading days (got {len(data)})"}
        if data.empty or "Close" not in data.columns:
            return {"error": "Invalid/missing price data"}
        
    
        analyzed = add_technical_indicators(data.copy())
        latest = analyzed.iloc[-1] 
        
        # Prepare results
        return {
            "symbol": symbol,
            "current_price": round(float(latest["Close"]), 2),
            "moving_averages": {
                "SMA_50": round(float(latest["SMA_50"]), 2),
                "SMA_200": round(float(latest["SMA_200"]), 2),
                "position": "Above 200DMA" if latest["Close"] > latest["SMA_200"] else "Below"
            },
            "momentum": {
                "RSI": round(float(latest["RSI"]), 1),
                "status": "Overbought (>70)" if latest["RSI"] > 70 
                          else "Oversold (<30)" if latest["RSI"] < 30 
                          else "Neutral"
            },
            "analysis_period": f"{analyzed.index[0].date()} to {analyzed.index[-1].date()}"
        }
        
    except Exception as e:
        return {"error": f"Analysis failed: {str(e)}"}
    
    
@tool("Risk Assessment Tool")
def risk_assessment_tool(symbol: str) -> dict:
    """Calculates stock risk metrics from Alpha Vantage data."""
    try:
        risk_data = get_risk_data(symbol)
        
        
        if isinstance(risk_data, dict) and "error" in risk_data:
            return risk_data
            
        
        df = pd.DataFrame(risk_data).T
        if df.empty:
            return {"error": "No price data available"}
            
        
        close_col = next((col for col in ["4. close", "close", "Close"] if col in df), None)
        if not close_col:
            return {"error": "Could not find closing prices"}
            
        close_prices = pd.to_numeric(df[close_col], errors='coerce').dropna()
        if len(close_prices) < 2:  
            return {"error": "Insufficient data points"}
            
        
        latest_close = round(float(close_prices.iloc[0]), 2)
        volatility = round(close_prices.pct_change().std(), 4)
        
        return {
            "symbol": symbol,
            "latest_price": latest_close,
            "volatility": volatility,
            "risk_level": "High" if volatility > 0.02 else "Medium" if volatility > 0.01 else "Low",
            "data_points": len(close_prices)
        }
        
    except Exception as e:
        return {"error": f"Risk assessment failed: {str(e)}"}
    


market_analyst = Agent(
    role="Senior Market Analyst",
    goal="Provide comprehensive technical analysis of stocks",
    backstory="""Expert in technical analysis with 20+ years experience analyzing
    stock patterns and market trends using advanced indicators""",
    llm=groq_llm_2,
    tools=[stock_analysis_tool],
    verbose=True 
)

risk_analyst = Agent(
    role="Chief Risk Officer",
    goal="Identify and quantify investment risks",
    backstory="""Former hedge fund risk manager specializing in volatility analysis
    and downside protection strategies""",
    llm=groq_llm_2,
    tools=[risk_assessment_tool],
    verbose=True  
)

#  Create Test Tasks
market_task = Task(
    description=f"Conduct complete technical analysis of {STOCK_SYMBOL}",
    agent=market_analyst,
    expected_output="""Concise report containing:
    1. Current price and key moving averages (50/200-day)
    2. RSI value and interpretation
    3. Basic trend classification"""
)

risk_task = Task(
    description="Fetch stock market data and risk assessment for the given stock symbol.",
    expected_output="A risk assessment report with stock price data and risk score.",
    agent=risk_analyst
    )

# Create and Run Test Crew
test_crew = Crew(
    agents=[market_analyst,risk_analyst],
    tasks=[market_task,risk_task],
    verbose=False 
)

# Execute 
try:
    print(f"\n🔍 Analyzing {STOCK_SYMBOL}...")
    result = test_crew.kickoff()
    print("\n📊 FINAL ANALYSIS REPORT:")
    print(result)
except Exception as e:
    print(f"\n❌ Error during analysis: {str(e)}")
    if "ALPHA_VANTAGE_API_KEY" in str(e):
        print("Please verify your Alpha Vantage API key in the .env file")


🔍 Analyzing TSLA...
# Agent: Senior Market Analyst
## Task: Conduct complete technical analysis of TSLA


# Agent: Senior Market Analyst
## Using tool: Stock Analysis Tool
## Tool Input: 
"{\"symbol\": \"TSLA\"}"
## Tool Output: 
{'symbol': 'TSLA', 'current_price': 261.31, 'moving_averages': {'SMA_50': 322.43, 'SMA_200': 286.64, 'position': 'Below'}, 'momentum': {'RSI': 62.3, 'status': 'Neutral'}, 'analysis_period': '2024-03-28 to 2025-03-28'}


# Agent: Senior Market Analyst
## Final Answer: 
## TSLA Technical Analysis

**Current Price & Moving Averages:**

* TSLA is currently trading at $261.31.
* The 50-day SMA is at $322.43, and the 200-day SMA is at $286.64.
The stock price is currently below both SMAs.

**RSI & Interpretation:**

* The RSI value is 62.3, indicating a neutral momentum condition.

**Trend Classification:**

* TSLA is currently experiencing a downtrend as the price is below both the 50-day and 200-day SMAs.


# Agent: Chief Risk Officer
## Task: Fetch stock market 

In [ ]:

from dotenv import load_dotenv
from crewai import Agent, Task, Crew, LLM
from crewai.tools import tool
import os
import requests
from typing import Dict, Any


load_dotenv()
STOCK_SYMBOL = "TSLA"  
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
groq_llm_2 = LLM(model="groq/gemma2-9b-it",max_tokens=700)

@tool("SerperSearchTool")
def serper_search_tool(query: str) -> Dict[str, Any]:
    """Searches the web using Serper API for the latest information.
    
    Args:
        query: The search query to look up
    """
    try:
        api_key = os.getenv("SERPER_API_KEY")
        if not api_key:
            return {"error": "Serper API key not found"}
            
        headers = {
            "X-API-KEY": api_key,
            "Content-Type": "application/json"
        }
        
        payload = {
            "q": query,
            "num": 3  
        }
        
        response = requests.post(
            "https://google.serper.dev/search",
            headers=headers,
            json=payload
        )
        
        search_results = response.json()
        
        
        formatted_results = {
            "query": query,
            "results": []
        }
        
        if "organic" in search_results:
            for result in search_results["organic"][:3]:
                formatted_results["results"].append({
                    "title": result.get("title"),
                    "link": result.get("link"),
                    "snippet": result.get("snippet")
                })
            
        return formatted_results
        
    except Exception as e:
        return {"error": f"Search failed: {str(e)}"}


def create_sentiment_analyst(llm):
    return Agent(
        role="Sentiment Analyst",
        goal="Analyze stock sentiment from latest news headlines",
        backstory="Expert AI analyst that quickly assesses market sentiment",
        llm=llm,
        verbose=True,
        tools=[serper_search_tool]
    )

def create_sentiment_task(agent):
    return Task(
        description=f"""
        Analyze sentiment for {STOCK_SYMBOL}'s stock from latest news headlines.
        Use SerperSearchTool to find recent articles and provide a concise analysis.
        Include sentiment direction and key reason for the assessment.
        """,
        agent=agent,
        expected_output="""
        {STOCK_SYMBOL} SENTIMENT: [Positive/Negative/Neutral]
        Strength: [Strong/Moderate/Weak]
        Key Point: [Brief reason in 10-15 words]
        Headline: "[Most significant recent headline]"
        """
    )


sentiment_analyst = create_sentiment_analyst(groq_llm_2)
sentiment_task = create_sentiment_task(sentiment_analyst)

# Create and Run Test Crew 
test_crew = Crew(
    agents=[sentiment_analyst],
    tasks=[sentiment_task],
    verbose=False 
)

# Execute with error handling
try:
    print(f"\n🔍 Analyzing {STOCK_SYMBOL}...")
    result = test_crew.kickoff()
    print("\n📊 FINAL ANALYSIS REPORT:")
    print(result)
except Exception as e:
    print(f"\n❌ Error during analysis: {str(e)}")
    if "ALPHA_VANTAGE_API_KEY" in str(e):
        print("Please verify your Alpha Vantage API key in the .env file")


🔍 Analyzing TSLA...
# Agent: Sentiment Analyst
## Task: 
        Analyze sentiment for TSLA's stock from latest news headlines.
        Use SerperSearchTool to find recent articles and provide a concise analysis.
        Include sentiment direction and key reason for the assessment.
        


# Agent: Sentiment Analyst
## Using tool: SerperSearchTool
## Tool Input: 
"{\"query\": \"TSLA latest news\"}"
## Tool Output: 
{'query': 'TSLA latest news', 'results': [{'title': 'Tesla, Inc. (TSLA) Stock Price, News, Quote & History - Yahoo Finance', 'link': 'https://finance.yahoo.com/quote/TSLA/', 'snippet': 'Find the latest Tesla, Inc. (TSLA) stock quote, history, news and other vital information to help you with your stock trading and investing.'}, {'title': 'Tesla, Inc. (TSLA) Latest Stock News & Headlines - Yahoo Finance', 'link': 'https://finance.yahoo.com/quote/TSLA/news/', 'snippet': "Tesla's 'American-made' cars won't get hit as hard by the auto tariffs. Just this week, Tesla boasted 